In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
# if using a Jupyter notebook, includue:
%matplotlib inline
import time
import keras
from enum import Enum
import keras
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras import layers
from math import floor, ceil
from random import sample

Using TensorFlow backend.


In [2]:
class CommentType(Enum):
    OTHER = 1
    WHITESPACE = 2
    PRAGMA = 3
    ENCODING = 4
    LICENSE = 5
    TYPEHINT = 6

class OtherType(Enum):
    MISC = 1
    RST = 2

class CleanAndCount:
    rst = 0
    punc = set(["+","-","=","_",">>",">>>","<<<","<<","*","'","\"","<",">","?","/","\\",",",".","!","@","#","$","%","^","&","(",")","[","]","{","}"])
    puncs = 0
    kinds = dict()
    def __init__(self):
        for k in CommentType:
            self.kinds[k] = 0
    def bumpkind(self,k):
        self.kinds[k] = self.kinds[k] + 1
    
    def sanitize_and_count(self,s):
        if s.isspace():
            self.bumpkind(CommentType.WHITESPACE)
            return None
        # TODO: Are there comments where c and c+'\n' are both in the data?
        s = s.strip()
        if (s.startswith("coding") or s.startswith("encoding")) and s.endswith("utf-8"):
            self.bumpkind(CommentType.ENCODING)
        elif (s.startswith("vim: ") or s.startswith("pylint: ") or s.startswith("noqa pylint: ") or s.startswith("pragma: ")):
            self.bumpkind(CommentType.PRAGMA)
        elif s.startswith("Licensed under "):
            self.bumpkind(CommentType.LICENSE)
        elif s.startswith("type:"):
            self.bumpkind(CommentType.TYPEHINT)
        else:
            self.bumpkind(CommentType.OTHER)
            if (".." in s and "::" in s):
                self.rst = self.rst + 1
            if (s in self.punc):
                self.puncs = self.puncs + 1
        return s
    # later should refactor to just clean and classify, so it can be reused for counting and filtering
    
    def __str__(self):
        return f"rst: {self.rst}, puncs: {self.puncs}, kinds: {(self.kinds)}"

class WordBox:
    def __init__(self, filename):
        self.filename = filename
        f = open(filename, "r", encoding="utf-8")
        lines = f.readlines()
        f.close()
        
        # Eventually the strings need to be embedded as integers, so we need to know what to do with them.  We need to know how many words there are, and we also need to know about outliers we can toss.
        # Note a subtlety with [str.split()](https://docs.python.org/3/library/stdtypes.html?highlight=split#str.split) below: passing space as the delimeter results in about 1.6 million empty splits. So it's important that we just use the general "consecutive whitespace" split default (no args). The original passed ```' '``` as the delimeter, which both lengthened lines and skewed distribution significantly.
        # find length of longest line, number of words total
        maxlen = 0
        wordcount = {'' : 0}
        for line in lines:
            words = line.split()
            maxlen = max(maxlen,len(words))
            for word in words:
                if word in wordcount:
                    wordcount[word] = wordcount[word] + 1
                else:
                    wordcount[word] = 1
        # Note that we haven't actually done further filtering; lots of these
        # "words" contain punctuation
        self.maxlen = maxlen
        self.wordcount = wordcount
        kvpairs = wordcount.items()
        self.kvsorted = sorted(kvpairs, key=lambda x: x[1], reverse=True)
        
        self.ndirect = len(lines)
        
        repeats = dict()
        for line in lines:
            if line in repeats:
                repeats[line] = repeats[line] + 1
            else:
                repeats[line] = 1
                
        self.repeats = repeats
        
        cac = CleanAndCount()
        cleaned = list()
        for l in lines:
            cleaned.append(cac.sanitize_and_count(l))
            
        # Let's see how many duplicate *cleaned* comments we have
        crepeats = dict()
        for line in cleaned:
            if line in crepeats:
                crepeats[line] = crepeats[line] + 1
            else:
                crepeats[line] = 1
        self.crepeats = crepeats
        
        cwordcount = {'' : 0}
        for line in cleaned:
            if line == None:
                continue
            words = line.split()
            maxlen = max(maxlen,len(words))
            for word in words:
                if word in cwordcount:
                    cwordcount[word] = cwordcount[word] + 1
                else:
                    cwordcount[word] = 1
        self.cwordcount = cwordcount
            
    def miscStats(self):
        vals = self.wordcount.values()
        print(len(vals))
        print(len(list(filter((lambda x: x > 1),vals))))
        print(len(list(filter((lambda x: x > 2),vals))))
        print(len(list(filter((lambda x: x > 3),vals))))
        print(len(list(filter((lambda x: x > 10),vals))))
        print(len(list(filter((lambda x: x > 100),vals))))
        print(str(self.ndirect)+" total lines")
        print(str(len(self.repeats))+" lines after removing exact duplicates")
        
        print("Raw comments: "+str((self.ndirect)))
        print("De-duplicated comments: "+str(len(self.repeats)))
        print("% duplicates: "+str(((self.ndirect-len(self.repeats))/self.ndirect)))

        print("De-duplicated *normalized* comments: "+str(len(self.crepeats)))
        print("% *normalized* duplicates: "+str(((self.ndirect-len(self.crepeats))/self.ndirect)))

        
    def plotTop(self):
        plt.hist(sorted(list(filter((lambda x: x > 100), self.wordcount.values()))),100,log=True)
    def showTopRepeats(self,n):
        print(sorted(self.repeats.items(), key=lambda x: x[1], reverse=True)[:n])
    def showTopCleanedRepeats(self,n):
        print(sorted(self.crepeats.items(), key=lambda x: x[1], reverse=True)[:n])

In [7]:
import os
import math

class Model:
    def __init__(self,wbox):
        self.sentences = []
        self.next_words = []
        self.maxlen = 10
        self.step = 2
        # Let's use the "cleaned" word counts (i.e., after deduplication)
        self.filtered = set(filter((lambda w: wbox.cwordcount[w] > 10), wbox.cwordcount.keys()))
        self.wbox = wbox
        # Choose a batch size depending on available physical RAM
        # memory calculation based on https://stackoverflow.com/questions/22102999/get-total-physical-memory-in-python/28161352#28161352
        # Based on number of bytes available, estimate how many inputs+outputs would fit in half of RAM
        # and take the next lowest power of 2 (the estimate is very rough) 
        #mem_bytes = os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES')
        #estimated_inputs = mem_bytes/2/(self.maxlen*10001*8) # maxlen * |dict| * |uint8|
        #self.batch_size = 2**(math.floor(math.log2(estimated_inputs)))
        #print("Chose batch size "+str(self.batch_size)+" based on "+str(mem_bytes/(1024.**3))+"GB RAM")
        # Using a huge batch size makes the training process very slow (fewer updates to weights per epoch)
        # Using small batch sizes makes the gradient estimate bad, and loss only increases.
        self.batch_size = 512 #256
        self.epochs = 35

    def fitTokens(self):
        print("Building dictionary from Keras Tokenizer at "+time.asctime())
        tokenizer = Tokenizer(num_words = 10000) # Note this will be much smaller than the 121K above... Could try len(words), but need to one-hot encode output
        #tokenizer = Tokenizer(num_words = len(words)) # Note this will be much smaller than the 121K above...

        dropNone = list(filter((lambda x: x != None), self.wbox.crepeats))
        
        # This shuffles the list randomly, such that any slice of the result is a valid random subsample.
        # This means we can split this 80/10/10 training/validation/test by range.
        dropNone = sample(dropNone, len(dropNone))

        tokenizer.fit_on_texts(dropNone)

        # One-hot takes gobs of memory, so.... can't do that
        #one_hot_results = tokenizer.texts_to_matrix(dropNone, mode='binary')
        word_index2 = tokenizer.word_index
        print("Dictionary built from Keras Tokenizer at "+time.asctime())
        print('Found %s unique tokens.' % len(word_index2))
        print("Fixing for the fact that num_words doesn't work")

        # per https://github.com/keras-team/keras/issues/8092#issuecomment-372833486
        tokenizer.word_index={e:i for e,i in tokenizer.word_index.items() if i <= 10000} # word index is sorted most frequent to least
        tokenizer.word_index[tokenizer.oov_token] = 10000+1
        self.word_index2 = tokenizer.word_index
        print('Trimmed to %s unique tokens.' % len(self.word_index2))

        print("Generating training data at "+time.asctime())
        # Let's only select training sets where all words are learned for now, to keep data small. This will cause issues later.
        self.sentences = []
        self.next_words = []
        discards = 0
        sentences_with_discards = 0
        for l in dropNone: # self.crepeats with the Nones removed
            words = l.split()
            localDiscard = False
            for i in range(-self.maxlen+1,len(words)-self.maxlen, self.step):
                #if all((w in self.word_index2) for w in words[i: i+self.maxlen+1]):
                # We'll train on anything that predicts a word in the dictionary,
                # regardless of how many 'missing' words are in the input
                if words[i+self.maxlen] in self.word_index2:
                    if i >= 0:
                        self.sentences.append(words[i: i+self.maxlen])
                    else:
                        # Use a dummy sequence that won't be in the dictionary; punctuation was already filtered out
                        lst = (("@#$ "*(-i))).split()
                        lst.extend(words[:i+self.maxlen])
                        self.sentences.append(lst)
                    self.next_words.append(words[i+self.maxlen])
                else:
                    localDiscard = True
                    discards = discards + 1
            if localDiscard == True:
                sentences_with_discards = sentences_with_discards + 1
        print("Original number of input sentences: ",len(self.wbox.crepeats))
        print('Number of usable sequences:', len(self.sentences))
        print("Sentences with discards: ",sentences_with_discards)
        print("Total sequences discarded: ", discards)
        print("Done generating training data at "+time.asctime())

        
    class SentenceSequence(keras.utils.Sequence):
        """A TensorFlow Sequence class for piece-wise training without allocating arrays for the full dataset at once. Trades space for time."""
        def __init__(self, batch_size, sentences, next_words, word_index, maxlen):
            self.batch_size = batch_size
            # Shuffle sentences randomly
            #self.sentences = random.sample(sentences, len(sentences))
            # TODO: Need to do simultaneous shuffle on sentences and words! Or, put code to generate next_words here, *after* shuffling sentences
            # TODO: actually, the shuffle should be done outside this sequence: I can use random.sample on the whole of sentences, and that's supposed to be adequate to ensure any subslice is a valid random sample (https://docs.python.org/3/library/random.html#random.sample): so sample the whole population, then split 80/20 or 90/10 for validation, and put the validation in a second generator. For that matter, should also split off testing data that way.
            self.sentences = sentences
            self.next_words = next_words
            self.word_index = word_index
            self.maxlen = maxlen
            
        def __len__(self):
            return ceil(len(self.sentences) / self.batch_size)
        
        def __getitem__(self, idx):
            # Because the output uses one-hot encoding, need to use |dictionary|+1 so the last value is undef token
            # It feels a bit wasteful to reallocate on every call instead of reusing memory, but really the alternative requires explicitly resetting lots of things to 0, so it's probably not a terrible penalty
            if (idx+1)*self.batch_size >= len(self.next_words):
                x = np.zeros((len(self.sentences) % self.batch_size, self.maxlen), dtype=np.int)
                y = np.zeros((len(self.sentences) % self.batch_size, len(self.word_index)+1), dtype=np.int)
            else:
                x = np.zeros((self.batch_size, self.maxlen), dtype=np.int)
                y = np.zeros((self.batch_size, len(self.word_index)+1), dtype=np.int)
            arrindex = 0 #idx * self.batch_size
            for i in range(idx * self.batch_size, min((idx + 1) * self.batch_size, len(self.next_words)-1)):
                try:
                    if (i < len(self.next_words)):
                        for t, word in enumerate(self.sentences[i]):
                            # Tweak: Since I'm dropping most words for scaling, need to handle 'unmapped' words
                            x[arrindex, t] = self.word_index.get(word, len(self.word_index))
                        y[arrindex, self.word_index.get(self.next_words[i],len(self.word_index))] = 1
                    arrindex = arrindex + 1
                except IndexError:
                    print("Encountered the bug")
                    print(i)
                    print(idx)
                    print(arrindex)
                    print(len(self.next_words))
                    print((idx + 1) * self.batch_size < len(self.next_words))
            # TODO: instead of reshaping, allocate correct shape initially and reuse
            # Need tensors of the actual shape above for input to the embedding layer!
            #self.x = self.x.reshape(self.x.shape[0], 10, 1)
            return (x, y)
            
    def buildGenerators(self,train,valid):
        lastTraining = ceil(train/100.0*len(self.sentences))
        firstValid = lastTraining + 1
        lastValid = ceil((train+valid)/100.0*len(self.sentences))
        self.train = self.SentenceSequence(self.batch_size, self.sentences[:lastTraining], self.next_words[:lastTraining], self.word_index2, self.maxlen)
        self.valid = self.SentenceSequence(self.batch_size, self.sentences[firstValid:lastValid], self.next_words[firstValid:lastValid], self.word_index2, self.maxlen)
        self.test = self.SentenceSequence(self.batch_size, self.sentences[lastValid+1:], self.next_words[lastValid+1:], self.word_index2, self.maxlen)

    def buildLayers(self):
        self.model = keras.models.Sequential()
        self.model.add(layers.Embedding((len(self.word_index2)+1),32, input_length=self.maxlen)) # TODO: Is the 10000 here the vocab size, or number of samples?
        #self.model.add(layers.Flatten())
        #model.add(layers.LSTM(32, input_shape=(10,1)))
        #self.model.add(layers.LSTM(32, input_shape=(10,len(self.word_index2)+1), return_sequences=True))
        #self.model.add(layers.LSTM(32))
        self.model.add(layers.GRU(32, return_sequences=True))
        self.model.add(layers.GRU(32)) #, return_sequences=True))
        self.model.add(layers.Dropout(0.2))
        #self.model.add(layers.Dense(len(self.word_index2), activation='softmax'))
        # TimeDistributed is useful if we want to predice *sequences* rather than just one word
        #self.model.add(layers.TimeDistributed(layers.Dense(len(self.word_index2))))
        self.model.add(layers.Dense(len(self.word_index2)+1, activation='softmax'))
        #self.model.add(layers.Activation('softmax'))
        self.model.summary()
        
    def fitModelWithGenerator(self):
        optimizer = keras.optimizers.Adamax(lr=0.01)
        # So we're training on loss against predicting the next word
        self.model.compile(loss='categorical_crossentropy', optimizer=optimizer,
                           metrics=['accuracy', 'categorical_accuracy'])
        self.model.summary()
        self.history = self.model.fit(self.train,
                           epochs=self.epochs,
                           #batch_size=self.batch_size, #32,
                           validation_data=self.valid)

In [8]:
import matplotlib.pyplot as plt

def plotLoss(m):
    plt.clf()
    loss = m.history.history['loss']
    val_loss = m.history.history['val_loss']

    epochs = range(1, len(loss)+1)

    loss = m.history.history['loss']
    val_loss = m.history.history['val_loss']
    epochs = range(1, len(loss) + 1)

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
        
def plotAccuracy(m):
    plt.clf()
    # no accuracy??? 
    acc = m.history.history['accuracy']
    val_acc = m.history.history['val_accuracy']
    epochs = range(1, len(acc)+1)
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    
def plotCatAccuracy(m):
    plt.clf()
    acc = m.history.history['categorical_accuracy']
    val_acc = m.history.history['val_categorical_accuracy']
    epochs = range(1, len(acc)+1)
    plt.plot(epochs, acc, 'bo', label='Training categorical acc')
    plt.plot(epochs, val_acc, 'b', label='Validation categorical acc')
    plt.title('Training and validation categorical accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Categorical Accuracy')
    plt.legend()
    plt.show()

In [10]:
def doItFlexibly(box,epochs):
    mod = Model(box)
    mod.epochs = epochs
    mod.fitTokens()
    mod.buildGenerators(70,15)
    mod.buildLayers()
    mod.fitModelWithGenerator()
    return mod
advancedBox = WordBox("commentdata2/advanced")
advancedM = doItFlexibly(advancedBox,100)
plotLoss(advancedM)

Building dictionary from Keras Tokenizer at Thu Aug 27 23:47:19 2020
Dictionary built from Keras Tokenizer at Thu Aug 27 23:47:25 2020
Found 105732 unique tokens.
Fixing for the fact that num_words doesn't work
Trimmed to 10001 unique tokens.
Generating training data at Thu Aug 27 23:47:25 2020
Original number of input sentences:  510767
Number of usable sequences: 2311439
Sentences with discards:  83864
Total sequences discarded:  101521
Done generating training data at Thu Aug 27 23:47:34 2020
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 10, 32)            320064    
_________________________________________________________________
gru_5 (GRU)                  (None, 10, 32)            6240      
_________________________________________________________________
gru_6 (GRU)                  (None, 32)                6240      
_______________

3161/3161 [==============================] - 295s 93ms/step - loss: 5.2493 - accuracy: 0.1861 - categorical_accuracy: 0.1861 - val_loss: 5.3453 - val_accuracy: 0.1881 - val_categorical_accuracy: 0.1881
Epoch 28/100
3161/3161 [==============================] - 295s 93ms/step - loss: 5.2469 - accuracy: 0.1864 - categorical_accuracy: 0.1864 - val_loss: 5.3043 - val_accuracy: 0.1872 - val_categorical_accuracy: 0.1872
Epoch 29/100
3161/3161 [==============================] - 295s 93ms/step - loss: 5.2438 - accuracy: 0.1865 - categorical_accuracy: 0.1865 - val_loss: 5.2972 - val_accuracy: 0.1885 - val_categorical_accuracy: 0.1885
Epoch 30/100
3161/3161 [==============================] - 295s 93ms/step - loss: 5.2413 - accuracy: 0.1867 - categorical_accuracy: 0.1867 - val_loss: 5.3598 - val_accuracy: 0.1884 - val_categorical_accuracy: 0.1884
Epoch 31/100
3161/3161 [==============================] - 295s 93ms/step - loss: 5.2387 - accuracy: 0.1869 - categorical_accuracy: 0.1869 - val_loss: 5.3

3161/3161 [==============================] - 295s 93ms/step - loss: 5.1949 - accuracy: 0.1895 - categorical_accuracy: 0.1895 - val_loss: 5.3312 - val_accuracy: 0.1884 - val_categorical_accuracy: 0.1884
Epoch 66/100
3161/3161 [==============================] - 295s 93ms/step - loss: 5.1934 - accuracy: 0.1897 - categorical_accuracy: 0.1897 - val_loss: 5.2890 - val_accuracy: 0.1887 - val_categorical_accuracy: 0.1887
Epoch 67/100
3161/3161 [==============================] - 295s 93ms/step - loss: 5.1937 - accuracy: 0.1895 - categorical_accuracy: 0.1895 - val_loss: 5.2857 - val_accuracy: 0.1890 - val_categorical_accuracy: 0.1890
Epoch 68/100
3161/3161 [==============================] - 295s 93ms/step - loss: 5.1929 - accuracy: 0.1896 - categorical_accuracy: 0.1896 - val_loss: 5.2961 - val_accuracy: 0.1880 - val_categorical_accuracy: 0.1880
Epoch 69/100
3161/3161 [==============================] - 294s 93ms/step - loss: 5.1923 - accuracy: 0.1897 - categorical_accuracy: 0.1897 - val_loss: 5.3

In [11]:
#plotAccuracy(advancedM)

Currently training only on sentences that are entirely within the top 10K words. Useful, but we'd like to also make predictions when some priming words are not mapped --- but we also don't want to have UNK be a valid prediction, because there will just be so many of them that it will heavily bias predictions towards UNK.
Advanced dataset loss hovering around 4.595, accuracy around 25%, when we can predict UNK and accept UNK inputs....

Now let's see now that we've switched to training on any inputs that predict some known word... (this will take longer to train since there are now many more valid training examples)

In [12]:
# TODO: currently hard-coded maxlen to 10...
# TODO: currently won't work unless there are exactly 10 words
def predictNext(model, primer):
    prime = primer.split()
    if (len(prime) > 10):
        return False
    # Will feed to batch prediction, with shape (None, maxlen)
    arr = np.zeros((1,10), dtype=np.int)
    for i in range(0,10):
        if prime[i] in model.word_index2:
            arr[0][i] = model.word_index2[prime[i]]
        else:
            arr[0][i] = len(model.word_index2)
    print(arr)
    return model.model.predict_on_batch(arr)
def maxVal(arr):
    tmp = 0
    for i in range(1,len(arr)):
        if arr[i] > arr[tmp]:
            tmp = i
    return tmp
def findWordFromIndexSlowly(model, idx):
    for key, value in model.word_index2.items():
        if value == idx:
            return key
    return False
def predictNextSplit(model, prime):
    # Will feed to batch prediction, with shape (None, maxlen)
    arr = np.zeros((1,model.maxlen), dtype=np.int)
    for i in range(0,model.maxlen):
        if prime[i] in model.word_index2:
            arr[0][i] = model.word_index2[prime[i]]
        else:
            arr[0][i] = len(model.word_index2)
    return model.model.predict_on_batch(arr)[0]
def next5(model, primer):
    prime = primer.split()
    if (len(prime) > model.maxlen):
        prime = prime[-10:]
    elif len(prime) < model.maxlen:
        prime = (("UNDEF "*(model.maxlen-len(prime)))+primer).split()
    orig = list()
    for i in range(0,5):
        w = findWordFromIndexSlowly(model, maxVal(predictNextSplit(model, prime)))
        prime.append(w)
        orig.append(prime.pop(0))
    orig.extend(prime)
    while (orig[0] == 'UNDEF'):
        orig.remove('UNDEF')
    return ' '.join(orig)

In [13]:
#print(advancedM.word_index2)
#print(next5(advancedM, "the method repeats"))
#print(next5(advancedM, "the method iterates over"))
#print(next5(advancedM, "sort the"))
#print(next5(advancedM, "the same"))
#print(next5(advancedM, "print the first parameter"))
#print(next5(advancedM, "standard input"))

In [14]:
#rawBox = WordBox("commentdata/raw")
#rawM = doItFlexibly(rawBox,100)
#plotLoss(rawM)

In [15]:
#basicBox = WordBox("commentdata/basic")
#basicM = doItFlexibly(basicBox,100)
#plotLoss(basicM)

In [16]:
#javarawBox = WordBox("commentdata/javaRaw")
#javarawM = doItFlexibly(javarawBox,100)
#plotLoss(javarawM)

In [17]:
#javacleanBox = WordBox("commentdata/javaCleaned")
#javacleanM = doItFlexibly(javacleanBox,100)
#plotLoss(javacleanM)

In [18]:
#next5(advancedM, "the method repeats")

In [19]:
#rawM.name = "Raw Python"
#basicM.name = "Basic Python"
advancedM.name = "Advanced Python"
#javarawM.name = "Raw Java"
#javacleanM.name = "Clean Java"

In [21]:
import pickle
models = [advancedM] #[rawM, basicM, advancedM] #, javarawM, javacleanM]
def predictAll(str):
    for m in models:
        print(m.name+": "+next5(m, str))
for m in models:
    prefix = m.name.replace(' ','_')
    m.model.save(prefix+"_shortgru.h5") #, save_format='h5')
    with open(prefix+"_shortgru.dict", "wb") as handle:
        pickle.dump(m.word_index2, handle)
    print('saved '+m.name)

saved Advanced Python


In [25]:
for m in models:
    print(m.name)
    m.eval = m.model.evaluate(m.test) #, batch_size=m.batch_size)

Advanced Python
678/678 [==============================] - 35s 52ms/step


In [26]:
advancedM.eval

[5.775980472564697, 0.18741382658481598, 0.18741382658481598]

In [ ]:
#predictAll("the method repeats")

In [ ]:
#predictAll("method iterates")

In [ ]:
#predictAll("sorts the list and")

In [ ]:
#predictAll("")

In [ ]:
#predictAll("sort the array")

In [ ]:
#predictAll("getter")

In [ ]:
#predictAll("class")

In [ ]:
#predictAll("throws an exception")

In [ ]:
#predictAll("thows an exception if")

In [ ]:
#predictAll("throws an exception if")

In [ ]:
#predictAll("if an error is encountered")

In [ ]:
#predictAll("logs all")

In [ ]:
#for m in models:
#    prefix = m.name.replace(' ','_')
#    m.model.save(prefix+"_shortgru.h5", save_format='h5')
#    with open(prefix+"_shortgru.dict", "wb") as handle:
#        pickle.dump(m.word_index2, handle)
#    print('saved '+m.name)

In [ ]:
class PredictionsOnly:
    def __init__(self, d, m):
        self.word_index2 = d
        self.model = m

def loadModel(prefix):
    d = Nil
    with open(prefix+"_shortgru.dict", "rb") as handle:
        d = pickle.load(handle)
    m = keras.models.load_model(prefix+"_shortgru.h5", save_format='h5')
    return PredictionsOnly(d, m)

In [ ]:
#tmp = loadModel("Advanced_Python")
#next5(tmp, "throws an exception if")

In [28]:
with open("testPhrases","r") as p:
    phrases = p.readlines()

for m in models:
    prefix = m.name.replace(' ','_')
    with open(prefix+"_gen.txt","w") as out:
        for p in phrases:
            s = next5(m,p)
            print(s)
            out.write(s+"\n")

sets the background color to the same number of
notifications regarding tribler in the home assistant config file
builds a localization loss for the test of the
b x t x y y y y y
ok to pass value in the list of the
the form should not be used to use the
import scansingleapi scanpostmancollection and the test is a
verify pricing file is not a single date and
issue using a bzfile function to be a string
we use get instead of the first line of
upload the cover thumbnail from the current window and
ensure that thresholded metrics are not in the same
latin small letter i a acute bsd license bsd
this endpoint is responsible for the given function param
test that biluocompatible tags are not in the same
a vfs that returns the path to the file
chunking not supported fall back to the same function
standard export converters to the base class for
removes from sprite group to be used for the
these derivers are not used in the case of
this method will always be a list of the
grammar example using 

remember that set members to the same name as
removing the top and bottom to the right time
logdebugno prefs file r and we need to check
returns character polish identity and the user is specified
this is an ellipse to compute the number of
an illegal configuration file is not a string or
returns the dtype for the given value of the
estimate the ideal throughput to the same time for
summary create new license file and the user is
create and return a list of the given name
keystroke a press of the given page the current
map asin codes to the same value as the
spec of the extendee is a list of files
special case for file and the file name and
not sure what we need to check the default
ok other merges into the new data in the
cosacosb can be combined with a single value of
which attributes to update the value of the given
list slicr configurations and check for the given name
manually import this hidden file and the file is
instruction movlpd groups sse sensor sensor for the given
handle 

todo this was copied to the same as the
test with placeholders no longer exist in the same
return npexpx pi with a new file in
testrecords frappegettestrecordsquality inspection of the given function returns
common implementation of cabortconnection and the following tests for
use path from entry to avoid a warning if
deenumerate the result is a list of the
temp breakpoint names comma separated list of the given
returns the height and height of the given position
if there was a new one of the the
r return the identity matrix with the number of
next we build the list of the files in
reverse the hlines to avoid the test for the
runorder of means stop the user and the user
wrapper around traceenumsampleelbo to be used to use the
container the container of the given name of the
inprocess task scheduler with the same time in the
return an iterator over the the given value in
delegate sign in and the to the given value
get the material group to be used to use
check that we handle the same va

intern some strings in the case of the
return a new bits for the given key and
having the dict keys in the list of the
versionadded an analogue of the given name and the
flush the internal buffer to the current time and
consider whether expr is a string of the string
iterate over self examples and the model and the
lookup this grain to be used to use
asked for specific amount of time to be used
test if target temperature is not in the same
call workflows entry functionmethod is a single server or
draw only keypoints locationnot into the given list of
a message that this is a simple test that
user remarks is not a list of the given
if we are not a single line of the
the default precision for the number of times the
set the maximum total number of the times the
support for niko home assistant component component for the
fit estimator and transform the model to the same
use the templates to be used to use the
add self and other values to the same variable
handle the case when the user is 

test with check mode for the given file and
these classes appear when the model is a list
now test that the test is not a list
sign not allowed in the same order as
create a user who is a user and the
this is to prevent a test that is not
hack perform surgery on the same time in the
center window on the top of the rect the
add function constructor to the same function as
if retval was already specified key name str str
surrounding space is optional in the list of the
from ruamelutilnew import convertversion and the test is not
get the latest data from the given file and
every module takes a string to the same type
adds values to the given list of the values
add add a register to the given function for
attempt to transform to a string with a string
traverse symbols in order to be used to use
mayavi plots require the same time for the model
with single byte string and the value is a
set second attribute to the given value if the
construct the rank bundle for the given name and
examples s

auto generated concat when the user is not a
remove an address from the given file and return
r returns the coleman of the given value of
only used if cacerts is not a string or
remove nodedefault and edgedefault the current state of the
string stuff for automatic the value of the given
raised when a master is specified the user is
normal gh gh not on the same as the
do provide keeptokens argument to the default value to
helper for total y and the data of the
f test joint hypothesis with the matrix of the
indexing test cases with a single line of
prevents the variable interfering on the same file as
none of the parameters in the list of the
add sighashall for testing the user to be used
on some platform stblocks and the user is not
always include weblate backend names in the given directory
list list of identifier names for the given name
return a platform for the given file name param
remove existing entry from the given file and
maybe it doesnt have a new path to the
tests of reduced